In [2]:
# 📦 Paso 1: Cargar librerías necesarias
import json
import os
import numpy as np
%pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'tensorflow'

  Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\javie\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-13e2df515630b4a41f92893938845698.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\javie\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# 🗂️ Paso 2: Cargar etiquetas desde el JSON
with open("resultados_dientes.json") as f:
    data = json.load(f)

# Convertir a listas
imagenes = []
etiquetas = []

for nombre, atributos in data.items():
    path = os.path.join("src", nombre)  # Ruta relativa a las imágenes
    if os.path.exists(path):
        imagenes.append(path)
        etiquetas.append(atributos["salud"])


In [ ]:
# 🖼️ Paso 3: Cargar imágenes y convertir a arrays
X = []
for path in imagenes:
    img = load_img(path, target_size=(128, 128))  # Redimensionamos
    img_array = img_to_array(img) / 255.0         # Normalizamos
    X.append(img_array)

X = np.array(X)
y = np.array(etiquetas)
print(f"Imágenes cargadas: {X.shape}, Etiquetas: {y.shape}")


In [ ]:
# ✂️ Paso 4: Dividir en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 🧠 Paso 5: Definir la CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Para clasificación binaria
])


In [ ]:
# ⚙️ Paso 6: Compilar y entrenar
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# 📈 Paso 7: Evaluar y visualizar resultados
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Evaluación final
loss, acc = model.evaluate(X_test, y_test)
print(f"Precisión final: {acc:.2f}")
